In [1]:
import pangolin
import g2o

In [19]:
import numpy as np

import time
import os 
from pathlib import Path
from numbers import Number
import re
import g2o
from PIL import Image
import g2o


def KeyPoints2CameraCoords(kp, K, w, h):
    # Keypoints to homogeneous point coordinates
    kp_tf = np.concatenate((kp, np.ones((np.shape(kp)[0],1))), axis=1)
    # Keypoints to camera coordinate system
    kp_tf = kp_tf @ np.linalg.inv(K.T)
    limits = np.squeeze(np.array([w, h, 1]) @ np.linalg.inv(K.T)) # e.g. [1920,1080,1] to camera coords
    w_k, h_k = limits[0,0], limits[0,1]
    # Define T_norm based on width and height of the image (now in camera coords)
    T_norm = np.diag([2/(w_k - 1.), 2/(h_k - 1.), 1]) # normalize
    T_norm[:,2] = [-1, -1, 1] # translate
    # Conditioning/Normalization of keypoints
    kp_tf = kp_tf @ T_norm.T
    return kp_tf, T_norm


def NormalizeKeypoints(kp, w, h):
    # Keypoints to homogeneous point coordinates
    kp_tf = np.concatenate((kp, np.ones((np.shape(kp)[0],1))), axis=1)
    # Define T_norm based on width and height of the image (now in camera coords)
    T_norm = np.diag([2/(w - 1.), 2/(h - 1.), 1]) # normalize
    T_norm[:,2] = [-1, -1, 1] # translate
    # Conditioning/Normalization of keypoints
    kp_tf = kp_tf @ T_norm.T
    return kp_tf, T_norm



    
class FeatureExtractor:
    def __init__(self):
        self.extractor = cv2.SIFT_create()
        
    def compute_features(self, img):
        pts = cv2.goodFeaturesToTrack(np.mean(img, axis=2).astype(np.uint8), 3000, qualityLevel=0.01, minDistance=7)
        kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=20) for f in pts]
        kp, des = self.extractor.compute(img, kps)
        return kp, des
        
        #kp, des = self.extractor.detectAndCompute(img,None)
        #return kp, des


class FeatureMatcher:
    def __init__(self):
        self.matcher = cv2.BFMatcher()
    def match_features(self, frame_prev, frame_cur):
        kp1, desc1 = frame_prev.keypoints, frame_prev.features
        kp2, desc2 = frame_cur.keypoints, frame_cur.features
        # Match descriptors.
        matches = self.matcher.knnMatch(desc1,desc2,k=1)
        # Sort the matches according to nearest neighbor distance ratio (NNDR) (CV course, exercise 4)
        distmat = np.dot(desc1, desc2.T)
        X_terms = np.expand_dims(np.diag(np.dot(desc1, desc1.T)), axis=1)
        X_terms = np.tile(X_terms,(1,desc2.shape[0]))
        Y_terms = np.expand_dims(np.diag(np.dot(desc2, desc2.T)), axis=0)
        Y_terms = np.tile(Y_terms,(desc1.shape[0],1))
        distmat = np.sqrt(Y_terms + X_terms - 2*distmat)
        ## We determine the mutually nearest neighbors
        dist1 = np.amin(distmat, axis=1)
        ids1 = np.argmin(distmat, axis=1)
        dist2 = np.amin(distmat, axis=0)
        ids2 = np.argmin(distmat, axis=0)
        pairs = []
        for k in range(ids1.size):
            if k == ids2[ids1[k]]:
                pairs.append(np.array([k, ids1[k], dist1[k]]))
        pairs = np.array(pairs)
        # We sort the mutually nearest neighbors based on the nearest neighbor distance ratio
        NNDR = []
        for k,ids1_k,dist1_k in pairs:
            r_k = np.sort(distmat[int(k),:])
            nndr = r_k[0]/r_k[1]
            NNDR.append(nndr)

        id_nnd = np.argsort(NNDR)
        return np.array(matches)[id_nnd]

class TransformationEstimator:
    # Uses ransac algorithm to find best estimation of essential matrix between images
    def __init__(self, N=10, Threshold=0.2):
        self.N_iterations = N
        self.Threshold = Threshold
        
    def F_from_point_pairs(xs,xss):
        # xs, xss: Nx3 homologous point coordinates, N>7
        # returns F: 3x3 Fundamental matrix
        # Coefficient matrix
        N = np.size(xs)[0]
        A = np.zeros((N, 9))
        for n in range(N):
            A[n,:] = np.kron(xss[n,:], xs[n,:])
        # Singular-value-decomposition
        U,D,V = np.linalg.svd(A, full_matrices=True, compute_uv=True)
        Fa = np.reshape(V[:,:,8], (3,3)) # approximation of F, could be > rank 2
        Ua,Da,Va = np.linalg.svd(Fa, full_matrices=True, compute_uv=True)
        F = (Ua * np.diag([Da[0,0], Da[1,1], 0])) @ Va
        return F 
    
    def E_from_point_pairs(xs,xss):
        # xs, xss: Nx3 homologous point coordinates, N>7
        # returns E: 3x3 Essential matrix
        # Coefficient matrix
        N = np.size(xs)[0]
        A = np.zeros((N, 9))
        for n in range(N):
            A[n,:] = np.kron(xss[n,:], xs[n,:])
        # Singular-value-decomposition
        U,D,V = np.linalg.svd(A, full_matrices=True, compute_uv=True)
        Ea = np.reshape(V[:,:,8], (3,3)) # approximation of F, could be > rank 2
        Ua,Da,Va = np.linalg.svd(Ea, full_matrices=True, compute_uv=True)
        E = (Ua * np.diag([1,1,0])) @ Va
        return E
      
    def ransack(self, cur_frame_kp, prev_frame_kp, matches, T_norm, K):
        # cur_frame_kp, prev_frame_kp: Nx2 pixel coordinates of keypoints matching between images
        # T_norm, 3x3 transformation matrix of keypoints: center of mass to (0,0), x and y to scale [-1,1]
        # K: camera calibration matrix
        N = np.size(cur_frame_kp)[0]
        highest_number_of_inliers = -1
        best_essential = np.eye(3)
        seed_n = 8 # has to be > 7
        # Keypoints to homogeneous point coordinates
        kp1 = np.concatenate((prev_frame_kp, np.ones(np.shape(N))))
        kp2 = np.concatenate((cur_frame_kp, np.ones(np.shape(N))))
        # Keypoints to camera coordinate system
        kp1 = kp1 @ K.T
        kp2 = kp2 @ K.T
        # Conditioning/Normalization of keypoints
        kp1 = kp1 @ T_norm.T
        kp2 = kp2 @ T_norm.T
        # Ransac loop
        for n in range(self.N_iterations):
            # Randomly select a seed group of > 7 matches
            seed_group = np.random.randint(low=0, high=N, size=seed_n)
            xs = kp1[seed_group,:]
            xss = kp2[seed_group,:]
            E = self.E_from_point_pairs(xs, xss)
        
        return(E)

class Frame:
    def __init__(self, rgb_fp, d_path, feature_extractor):
        self.rgb = cv2.imread(rgb_fp)
        # depth file read is handled bit differently
        depth = cv2.imread(d_path)
        self.depth =  Image.open(d_path)
        self.keypoints, self.features  = None, None
        self.feature_extractor = feature_extractor
    def process_frame(self):
        self.keypoints, self.features = self.feature_extract(self.rgb)
        return self.keypoints, self.features, self.rgb
        
    def feature_extract(self, rgb):
        return self.feature_extractor.compute_features(rgb)
        
def compute_fundamental_matrix(kp1, kp2, matches):
    """
    Takes in filenames of two input images 
    Return Fundamental matrix computes 
    using 8 point algorithm
    """
    
    # extract points
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    
    # Compute fundamental matrix
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_8POINT)
    return F 

def essentialToRt(E, opt = 1):
    # see wikipedia: https://en.wikipedia.org/wiki/Essential_matrix
    U,d,Vt = np.linalg.svd(E)
    W = np.mat([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    Winv = W.T
    # ansatz (educated guess)
    #t = U @ W @ np.diag(d) @ U.T
    #t = Vt[-1,:]
    R = U @ Winv @ Vt
    Z = np.mat([[0, 1, 0], [-1, 0, 0], [0, 0, 0]]) 
    if opt==1:
        t = U @ Z @ U.T
        t = np.array([t[2,1], t[0,2], t[1,0]]) # get vector from transpose operation
    elif opt==2:
        t = U @ W @ np.diag(d) @ U.T
        t = np.array([t[2,1], t[0,2], t[1,0]]) 
    else:
        t = Vt[-1,:]
    return R, t

debug = True
scale = 5000
D = np.array([0, 0, 0, 0], dtype=np.float32)  # no distortion
K = np.matrix([[481.20, 0, 319.5], [0, 480.0, 239.5], [0, 0, 1]])  # camera intrinsic parameters
fx, fy, cx, cy = 481.20, 480.0, 319.5, 239.5

width, height = 640, 480

def get_color(img, pt):
        x = int(np.clip(pt[0], 0, width - 1))
        y = int(np.clip(pt[1], 0, height - 1))
        color = img[y, x]
        if isinstance(color, Number):
            color = np.array([color, color, color])
        return color[::-1] / 255.

def point2dTo3d(n, m, d):
    z = float(d) / scale
    x = (n - cx) * z / fx
    y = (m - cy) * z / fy
    point = np.array([x, y, z], dtype=np.float32)
    return point


def solvePnP(frame1, frame2, matches):
        kp1, kp2, des1, des2, depth = frame1.keypoints, frame2.keypoints, frame1.features, frame2.features, frame1.depth
        goodMatches = matches
    
        pts_obj, pts_img2, pts_img1 = [], [], []
        colour = []
        features = []
        for i in range(0, len(goodMatches)):
            p = kp1[goodMatches[i][0].queryIdx].pt
            # d = depth[int(p[1])][int(p[0])]
            d = depth.getpixel((int(p[0]), int(p[1])))
            if d == 0:
                pass
            else:
                p2 = kp2[goodMatches[i][0].trainIdx].pt
                #dif = abs(cv2.norm(p) - cv2.norm(p2))
                #if dif > .1:
                    #print('dif -> {}'.format(dif))
                    #pass
                pts_img2.append(p2)
                pts_img1.append(p)
                pd = point2dTo3d(p[0], p[1], d)
                pts_obj.append(pd)
                # c = frame1.rgb[int(p[1])][int(p[0])]
                colour.append(get_color(img=frame1.rgb, pt=p))
                features.append(des1[goodMatches[i][0].queryIdx])

        pts_obj, pts_img2 = np.array(pts_obj), np.array(pts_img2)
        pts_img1 = np.array(pts_img1)
        if debug:
            print('pts_obj -> {}, pts_img->{}'.format(np.shape(pts_obj), np.shape(pts_img2)))
            print(np.shape(cv2.solvePnPRansac(pts_obj, pts_img2, K, D, useExtrinsicGuess=False)))
        
        retval, rvec, tvec, inliers = cv2.solvePnPRansac(pts_obj, pts_img2, K, D, useExtrinsicGuess=False)
        return retval, rvec, tvec, inliers

def transformMatrix(rvec, tvec):
        r, t = np.matrix(rvec), np.matrix(tvec)
        R, _ = cv2.Rodrigues(r)
        Rt = np.hstack((R, t))
        T = np.vstack((Rt, np.matrix([0, 0, 0, 1])))
        return T


def opencv_R_t(E,kp1, kp2, matches):
    """ 
    Return Fundamental matrix computes 
    using 8 point algorithm
    """
    
    # extract points
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    
    pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K, distCoeffs=None)
    pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K, distCoeffs=None)

    points, R, t, mask = cv2.recoverPose(E,pts_l_norm, pts_r_norm)
    return R,t


def triangulation(kp1, kp2, T_1w, T_2w):
    """Triangulation to get 3D points
    Args:
        kp1 (Nx2): keypoint in view 1 (normalized)
        kp2 (Nx2): keypoints in view 2 (normalized)
        T_1w (4x4): pose of view 1 w.r.t  i.e. T_1w (from w to 1)
        T_2w (4x4): pose of view 2 w.r.t world, i.e. T_2w (from w to 2)
    Returns:
        X (3xN): 3D coordinates of the keypoints w.r.t world coordinate
        X1 (3xN): 3D coordinates of the keypoints w.r.t view1 coordinate
        X2 (3xN): 3D coordinates of the keypoints w.r.t view2 coordinate
    """
    kp1_3D = np.ones((3, kp1.shape[0]))
    kp2_3D = np.ones((3, kp2.shape[0]))
    kp1_3D[0], kp1_3D[1] = kp1[:, 0].copy(), kp1[:, 1].copy()
    kp2_3D[0], kp2_3D[1] = kp2[:, 0].copy(), kp2[:, 1].copy()
    X = cv2.triangulatePoints(T_1w[:3], T_2w[:3], kp1_3D[:2], kp2_3D[:2])
    X /= X[3]
    X1 = T_1w[:3] @ X
    X2 = T_2w[:3] @ X
    return X[:3], X1, X2 


def estimate_relative_pose_from_correspondence(pts1, pts2, K1, K2):
        f_avg = (K1[0, 0] + K2[0, 0]) / 2
        pts1, pts2 = np.ascontiguousarray(pts1, np.float32), np.ascontiguousarray(pts2, np.float32)

        pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K1, distCoeffs=None)
        pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K2, distCoeffs=None)

        E, mask = cv2.findEssentialMat(pts_l_norm, pts_r_norm, focal=1.0, pp=(0., 0.),
                                       method=cv2.RANSAC, prob=0.999, threshold=3.0 / f_avg)
        points, R_est, t_est, mask_pose = cv2.recoverPose(E, pts_l_norm, pts_r_norm)
        return mask[:,0].astype(np.bool), R_est, t_est 


def estimateEssential(kp1, kp2, matches, K, essTh = K[0,0]):
    #f_avg = K[0,0]
    # match keypoints
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    # normalize points
    pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K, distCoeffs=None)
    pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K, distCoeffs=None)
    E, inliers = cv2.findEssentialMat(pts_l_norm, pts_r_norm, focal=1.0, pp=(0., 0.),
                                    method=cv2.RANSAC, prob=0.999, threshold=3.0 / essTh)
    return E, inliers
        


def estimateHomography(kp1, kp2, matches, K, homTh=K[0,0]):
    # match keypoints
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    # normalize points
    pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K, distCoeffs=None)
    pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K, distCoeffs=None)
    
    H, inliers = cv2.findHomography(pts_l_norm, pts_r_norm, cv2.RANSAC, ransacReprojThreshold=homTh)
    return H, inliers


def estimateRelativePose(tform, kp1, kp2, matches, K, tform_type = "Homography"):
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    # normalize points
    pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K, distCoeffs=None)
    pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K, distCoeffs=None)
    
    if tform_type == "Homography":
        num, Rs, Ts, Ns  = cv2.decomposeHomographyMat(tform, K)
        R, t, validFraction = chooseRealizableSolution(Rs, Ts, K, pts_l_norm, pts_r_norm)
        return R, t
        
    elif tform_type == "Essential":
        points, R, t, _ = cv2.recoverPose(tform, pts_l_norm, pts_r_norm)    
        return R, t
    else:
        print("Unknown transformation type")
        
        
import numpy as np

# TODO: replace all 

def triangulateMidPoint(points1, points2, P1, P2):
    points1 = np.squeeze(points1)
    points2 = np.squeeze(points2)
    numPoints = points1.shape[0]
    points3D = np.zeros((numPoints,3))
    P1 = P1.T
    P2 = P2.T
    M1 = P1[:3, :3]
    M2 = P2[:3, :3]
    # Get least-squares solution
    c1 = np.linalg.lstsq(-M1,  P1[:,3], rcond=None)[0]
    #print(np.shape(M1))
    c2 = np.linalg.lstsq(-M2, P2[:,3], rcond=None)[0]
    y = c2 - c1
    u1 = np.concatenate((points1, np.ones((numPoints,1))), axis=1)
    u2 = np.concatenate((points2, np.ones((numPoints,1))), axis=1)
    #u1 = [points1, ones(numPoints, 1, 'like', points1)]'
    #u2 = [points2, ones(numPoints, 1, 'like', points1)]'
    a1 = np.linalg.lstsq(M1, u1.T, rcond=None)[0]
    a2 = np.linalg.lstsq(M2, u2.T, rcond=None)[0]
    #isCodegen  = ~isempty(coder.target);
    condThresh = 2**(-52)
    for i in range(numPoints):
        A   = np.array([a1[:,i], -a2[:,i]]).T 
        AtA = A.T@A
        if np.linalg.cond(AtA) < condThresh: # original: rcond(AtA) < condThresh
            # Guard against matrix being singular or ill-conditioned
            p    = np.inf(3, 1)
            p[2] = -p[2]
        else:
            alpha = np.linalg.lstsq(A, y, rcond=None)[0]
            p = (c1 + (alpha[0] * a1[:,i]).T + c2 + (alpha[1] * a2[:,i]).T) / 2
            
        points3D[i, :] = p.T

    return points3D

    


def chooseRealizableSolution(Rs, Ts, K, points1, points2):
    # Rs is 4x3x3, holding all possible solutions of Rotation matrix
    # Ts is 4x3x1, holding all possible solutions of Translation vector
    numNegatives = np.zeros((np.shape(Ts)[0], 1))
    #  The camera matrix is computed as follows:
    #  camMatrix = [rotationMatrix; translationVector] * K
    #  where K is the intrinsic matrix.
    #camMatrix1 = cameraMatrix(cameraParams1, np.eye(3), np.array([0 0 0]));
    camMatrix1 = np.concatenate((np.eye(3), np.zeros((1,3))), axis=0) @ K
    
    for i in range(np.shape(Ts)[0]):
        #camMatrix2 = cameraMatrix(cameraParams2, Rs(:,:,i)', Ts(i, :));
        #camMatrix2 is 4x3 @ 3x3 matmul
        camMatrix2 = np.concatenate((Rs[i].T, Ts[i].T),axis=0) @ K
        m1 = triangulateMidPoint(points1, points2, camMatrix1, camMatrix2)
        #m2 = bsxfun(@plus, m1 * Rs(:,:,i)', Ts(i, :));
        m2 = (m1 @ (Rs[i]).T) + Ts[i].T
        numNegatives[i] = np.sum((m1[:,2] < 0) | (m2[:,2] < 0))
        
    val = np.min(numNegatives)
    idx = np.where(numNegatives==val)
    validFraction = 1 - (val / points1.shape[0])
    
    R = np.zeros((len(idx), 3,3))
    t = np.zeros((len(idx), 3))
    for n in range(len(idx)):
        idx_n = idx[n][0]
        R0 = Rs[idx_n].T
        t0 = Ts[idx_n].T

        tNorm = np.linalg.norm(t0)
        if tNorm != 0:
            t0 = t0 / tNorm
        R[n] = R0
        t[n] = t0

    return R, t, validFraction



In [20]:
# Filepaths
cur_dir = "/home/juuso"
dir_rgb = cur_dir + "/visual_slam/data/ICL_NUIM/rgb/"
dir_depth = cur_dir + "/visual_slam/data/ICL_NUIM/depth/"
is_WINDOWS = False
if is_WINDOWS:
    dir_rgb = dir_rgb.replace("/", "\\")
    dir_depth = dir_depth.replace("/", "\\")
# Initialize
feature_extractor = FeatureExtractor()
feature_matcher = FeatureMatcher()
# run feature extraction for 1st image
fp_rgb = dir_rgb + str(1) + ".png"
fp_depth = dir_depth + str(1) + ".png"
cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
kp, features, rgb = cur_frame.process_frame() 
prev_frame = cur_frame

for i in range(2,1000):
    if i % 30 == 0:
        fp_rgb = dir_rgb + str(i) + ".png"
        fp_depth = dir_depth + str(i) + ".png"
        # Feature Extraction for current frame
        cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
        kp, features, rgb = cur_frame.process_frame()
        # Feature Matching to previous frame
        matches = feature_matcher.match_features(prev_frame, cur_frame)    
        # if not enough matches (<100) continue to next frame
        if(len(matches) < 100):
            continue
        # TODO: compute homography and inliers
        H, inliersH  = estimateHomography(prev_frame.keypoints, cur_frame.keypoints, matches, K)
        # TODO: compute essential and inliers
        E, inliersE  = estimateEssential(prev_frame.keypoints, cur_frame.keypoints, matches, K)
        # TODO: choose between models based on number of inliers
        # https://www.programcreek.com/python/example/70413/cv2.RANSAC
        tform = H
        inliers = inliersH
        tform_type = "Homography"
        if sum(inliersH) < sum(inliersE):
            print("Chose Essential")
            tform = E
            inliers = inliersE
            tform_type = "Essential"
        else:
            print("chose Homography")
        # TODO: if number of inliers with the better model too low continue to next frame
        if sum(inliers) < 100:
            continue
        # TODO: get pose transformation
        r,t = estimateRelativePose(tform, prev_frame.keypoints, cur_frame.keypoints, matches, K, tform_type)
        print("Rotation")
        print(r)
        print("Translation")
        print(t)
        # TODO: triangulate two view to obtain 3-D map points
        
        
        # Display
        img3 = cv2.drawMatchesKnn(prev_frame.rgb,prev_frame.keypoints, cur_frame.rgb,cur_frame.keypoints,matches[:100],None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        #img2 = cv2.drawKeypoints(rgb, kp, None, color=(0,255,0), flags=0)
        cv2.imshow('a', img3)
        cv2.waitKey(0)
        #
        
        
        
        
        
        prev_frame = cur_frame

Chose Essential
Rotation
[[ 9.99504922e-01 -1.67506360e-02 -2.66331853e-02]
 [ 1.67488668e-02  9.99859686e-01 -2.89518124e-04]
 [ 2.66342979e-02 -1.56700885e-04  9.99645232e-01]]
Translation
[[0.31807891]
 [0.10384802]
 [0.94235948]]
chose Homography
Rotation
[[[ 0.20331852 -0.92008714  0.33481508]
  [-0.91678056 -0.29895901 -0.26483298]
  [ 0.34376571 -0.2531063  -0.90430199]]

 [[ 0.20331852 -0.92008714  0.33481508]
  [-0.91678056 -0.29895901 -0.26483298]
  [ 0.34376571 -0.2531063  -0.90430199]]]
Translation
[[-0.51165926 -0.38660982  0.76729241]
 [ 0.51165926  0.38660982 -0.76729241]]
chose Homography
Rotation
[[[-0.04083678 -0.76428572  0.25635332]
  [-0.18941529 -0.57295569  0.08691766]
  [ 0.54279238 -0.2949066  -0.78638341]]

 [[ 0.33133925 -0.37638697 -0.63270092]
  [ 0.40450204 -0.27245962 -0.36552442]
  [-0.24243779 -0.8851439   0.39716164]]]
Translation
[[-0.51095048 -0.38375825  0.76919387]
 [-0.51097986 -0.38375915  0.7691739 ]]
chose Homography
Rotation
[[[-0.81236261 -0.

In [ ]:
# Filepaths
cur_dir = "/home/juuso"
dir_rgb = cur_dir + "/visual_slam/data/ICL_NUIM/rgb/"
dir_depth = cur_dir + "/visual_slam/data/ICL_NUIM/depth/"
is_WINDOWS = False
if is_WINDOWS:
    dir_rgb = dir_rgb.replace("/", "\\")
    dir_depth = dir_depth.replace("/", "\\")
# Initialize
feature_extractor = FeatureExtractor()
feature_matcher = FeatureMatcher()


# run feature extraction for 1st image
fp_rgb = dir_rgb + str(1) + ".png"
fp_depth = dir_depth + str(1) + ".png"
cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
kp1, features1, rgb1 = cur_frame.process_frame() 

prev_frame = cur_frame
i = 30 # jump to 30th frame

fp_rgb = dir_rgb + str(i) + ".png"
fp_depth = dir_depth + str(i) + ".png"
# Feature Extraction for current frame
cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
kp2, features2, rgb2 = cur_frame.process_frame()

# Feature Matching to previous frame
matches = feature_matcher.match_features(prev_frame, cur_frame) 

best8 = matches[:8]

F = compute_fundamental_matrix(kp1, kp2, matches[:100])
F
#img3 = cv2.drawMatchesKnn(prev_frame.rgb,prev_frame.keypoints, cur_frame.rgb,cur_frame.keypoints,matches[:8],None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#img2 = cv2.drawKeypoints(rgb, kp, None, color=(0,255,0), flags=0)
#cv2.imshow('a', img3)
#cv2.waitKey(0)
E = K.T @ F @ K
#print(E)

print("Handcrafted")
R, t = essentialToRt(E)
t = np.array([t[2,1], t[0,2], t[1,0]])
print(R)
print(t)

print(np.linalg.norm(t))


print("opencv")

R, t = opencv_R_t(E, kp1, kp2, matches[:100])
print(R)
print(t)


#t = np.array([t_mat[2,1], t_mat[0,2], t_mat[1,0]])
#t = t/np.linalg.norm(t)
#print(np.linalg.norm(t))
# E = U@np.diag(S)@VT


print("Ground truth")
retval, rvec, tvec, inliers = solvePnP(prev_frame, cur_frame, matches)
T = transformMatrix(rvec, tvec)
print(T)
print(np.linalg.norm(tvec))



In [ ]:
#print(np.array(kp1))

# extract points
pts1 = []
pts2 = []
for i,(m) in enumerate(matches):
    #print(m.distance)
    pts2.append(kp2[m[0].trainIdx])
    pts1.append(kp1[m[0].queryIdx])

pts1  = np.asarray(pts1)
pts2 = np.asarray(pts2)


F, mask = cv2.findFundamentalMat(pts1[:100], pts2[:100], cv2.FM_8POINT)
print("x2.T @ F @ x1 = (should be close to 0)" )


test_idx = 20

x1 = np.expand_dims(pts1[test_idx,:], axis=1)
x2 = np.expand_dims(pts2[test_idx,:], axis=1)


#print(np.shape(x2))



print(x2.T @ F @ x1)

E = K.T @ F @ K # Get the essential matrix

[U,S,V] = np.linalg.svd(E)

diag_110 = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 0]])
newE = U@diag_110@V
[U,S,V] = np.linalg.svd(newE); # Perform second decompose to get S=diag(1,1,0)

W = np.array([[0, -1, 0],[1, 0, 0],[0, 0, 1]]) # [0 -1 0; 1 0 0; 0 0 1];

R1 = U@W@V
R2 = U@W.T@V
t1 = U[:,2] #norm = 1
t2 = -U[:,2] #norm = 1


#print(R1, t1)


In [ ]:
T_norm = np.diag([2/(1920 - 1.), 2/(1080 - 1.), 1]) # normalize
T_norm[:,2] = [-1, -1, 1] # translate

a = np.array([1920,1080,1]) @ np.linalg.inv(K.T)

np.shape(np.squeeze(a))

In [ ]:
print([m[0].queryIdx for m in best8])
[m[0].trainIdx for m in best8]